<a href="https://colab.research.google.com/github/QingfangLiu/ml-dl-notebooks/blob/main/VAEs/VAE_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### building vae using PyTorch from scratch
* using MNIST dataset

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
import torchvision.datasets as datasets
from tqdm import tqdm # progress bar
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

In [ ]:
class VariationalAutoEncoder(nn.Module): # inherits from nn.Module
    def __init__(self, input_dim, h_dim=200, z_dim=20):
        super().__init__() # initializes all internal PyTorch machinery in nn.Module
        # define basic layers
        # encoder
        self.img_2hid = nn.Linear(input_dim, h_dim)
        self.hid_2mu = nn.Linear(h_dim, z_dim)
        self.hid_2sigma = nn.Linear(h_dim, z_dim)

        # decoder
        self.z_2hid = nn.Linear(z_dim, h_dim)
        self.h_2img = nn.Linear(h_dim, input_dim)

        self.relu = nn.ReLU()

    # encoder and decoder are helper functions to make the code modular and clean
    def encode(self,x):
        # q_phi(z|x)

        h = self.relu(self.img_2hid(x))
        mu, sigma = self.hid_2mu(h), self.hid_2sigma(h) # mu and sigma come from linear layer, no need for relu
        return mu, sigma

    def decode(self,z):
        # p_theta(x|z)

        h = self.relu(self.z_2hid(z))
        return torch.sigmoid(self.h_2img(h)) # to bring scale back to 0-1

    # combine everything in forward function
    # defines the full model logic
    # by default being called when calling the whole class
    def forward(self,x):
        mu, sigma = self.encode(x)
        epsilon = torch.randn_like(sigma) # sample from N(0,1) with same shape as sigma
        z_reparametrized = mu + sigma * epsilon
        x_reconstructed = self.decode(z_reparametrized)
        return x_reconstructed, mu, sigma # need all 3 to calculate loss function

In [ ]:
# test the model
if __name__ == "__main__":
    x = torch.randn(1,28*28)
    vae = VariationalAutoEncoder(input_dim=784)
    x_reconstructed, mu, sigma = vae(x)
    print(x_reconstructed.shape)
    print(mu.shape)
    print(sigma.shape)

torch.Size([1, 784])
torch.Size([1, 20])
torch.Size([1, 20])


In [ ]:
# configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_DIM = 784
H_DIM = 200
Z_DIM = 20

NUM_EPOCHS = 10
BATCH_SIZE = 32
LR_RATE = 3e-4 # Karpathy constant


In [ ]:
# download MNIST dataset to local drive
dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 507kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.60MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.50MB/s]


In [ ]:
train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model = VariationalAutoEncoder(INPUT_DIM, H_DIM, Z_DIM).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR_RATE)
loss_fn = nn.BCELoss(reduction="sum")

In [ ]:
# start training
for epoch in range(NUM_EPOCHS):
    loop = tqdm(enumerate(train_loader)) # to get progress bar
    for i, (x, _) in loop:
        # forward pass
        x = x.to(DEVICE).view(x.shape[0], INPUT_DIM) # reshape input dimension (can also use reshape)
        x_reconstructed, mu, sigma = model(x)

        # compute loss
        reconstruction_loss = loss_fn(x_reconstructed, x)
        kl_div = -torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))

        # back prop
        loss = reconstruction_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())

1875it [00:14, 126.14it/s, loss=5.39e+3]
1875it [00:14, 130.30it/s, loss=4.88e+3]
1875it [00:16, 115.70it/s, loss=4.36e+3]
1875it [00:14, 130.79it/s, loss=4.28e+3]
1875it [00:14, 133.00it/s, loss=4.25e+3]
1875it [00:14, 132.57it/s, loss=4.18e+3]
1875it [00:14, 133.27it/s, loss=3.96e+3]
1875it [00:14, 127.30it/s, loss=4.1e+3]
1875it [00:14, 131.82it/s, loss=3.8e+3]
1875it [00:14, 132.90it/s, loss=4.06e+3]


In [ ]:
# how to do inference
def inference(digit, num_examples=1):
    images = []
    idx = 0
    for x, y in dataset:
        if y == idx:
            images.append(x)
            idx += 1
        if idx == 10:
            break

    encodings_digit = []
    for d in range(10):
        with torch.no_grad():
            mu, sigma = model.encode(images[d].view(1, 784))
        encodings_digit.append((mu, sigma))

    mu, sigma = encodings_digit[digit]
    for example in range(num_examples):
        epsilon = torch.randn_like(sigma)
        z = mu + sigma * epsilon
        out = model.decode(z)
        out = out.view(-1, 1, 28, 28) # PyTorch tensor method to reshape tensor (-1 is the placeholder to auto infer the dim)
        save_image(out, f"generated_{digit}_ex{example}.png")


In [ ]:
for idx in range(10):
    inference(idx, num_examples=5)